In [92]:
import pandas as pd

data = pd.read_csv('./data/iris.csv')
data

,150,4,setosa,versicolor,virginica
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [93]:
from sklearn.datasets import load_iris
import sklearn as sk
import numpy as np
from sklearn.decomposition import PCA

iris = load_iris()
print(type(iris))
X = iris.data # 四维特征
y = iris.target # 标签（类）

# 将X, y水平拼接 y.reshape(-1, 1) 将y转换为列向量(y原来是行向量(150,))
X_y = np.concatenate((X, y.reshape(-1, 1)), axis=1)

# print(X)
print(y)
print(X_y[0])
print(X_y.shape)

# 对数据做pca降维指定维度
pca = PCA(n_components=2)

# 对X做pca降维
X_pca = pca.fit_transform(X)
# 将X_pca, y水平拼接
X_pca_y = np.concatenate((X_pca, y.reshape(-1, 1)), axis=1)
print(X_pca_y.shape)
print(X_pca_y[0])

<class 'sklearn.utils._bunch.Bunch'>
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
[5.1 3.5 1.4 0.2 0. ]
(150, 5)
(150, 3)
[-2.68412563  0.31939725  0.        ]


# 对四个特征pca降维为2后再使用两个权重参数一个bias来拟合数据，最终结果采用pca降低特征维度后展示

使用交叉验证，将所有数据集划分为5份，其中4份用于训练，1份用于测试，轮换后最终得到5个测试结果，取平均值作为最终结果。

In [94]:
# %pdb # 触发assert报错时，自动进入调试模式
# dataset 
## 读取sklearn自带的数据集，变量类型为sk.utils.Bunch
class Dataset:
    def __init__(self, data: sk.utils.Bunch):
        
        
        # 对数据做pca降维指定维度
        pca = PCA(n_components=2)
        X = data.data # 四维特征
        y = data.target # 标签（类）
        
        # 对X做pca降维
        X_pca = pca.fit_transform(X)

        # 将data.data和data.target随机打乱切分为五份，每份的比例为0.2，并将data.data和data.target水平拼接
        temp_data = np.concatenate((X_pca, y.reshape(-1, 1)), axis=1)
        random_data = np.random.permutation(temp_data) # 将data.data随机打乱
        # random_data插入倒数第二列，插入的值为1，用于bias
        random_data = np.insert(random_data, random_data.shape[1]-1, 1, axis=1)
        assert random_data.shape == (150, 4)
        assert random_data[0][-2] == 1
        self.data = random_data # 150*4
        # 将数据集切分为五份，每份的比例为0.2
        self.train_idx = int(0.8 * data.data.shape[0])
        self.train_feature = self.data[0:self.train_idx, 0:-1]
        assert self.train_feature.shape == (120, 3)
        self.train_label = self.data[0:self.train_idx, -1]
        assert self.train_label.shape == (120,)
        self.test_feature = self.data[self.train_idx:, 0:-1]
        assert self.test_feature.shape == (30, 3)
        self.test_label = self.data[self.train_idx:, -1]
        assert self.test_label.shape == (30,)
    def random(self):
        self.data = np.random.permutation(self.data)
        self.train_feature = self.data[0:self.train_idx, 0:-1]
        self.train_label = self.data[0:self.train_idx, -1]
dataset = Dataset(iris)
print(dataset.train_feature.shape)
print(dataset.train_label.shape)
del dataset

(120, 3)
(120,)


## softmax regression

反向传播的公式：

$$
\begin{align*}
&\frac{\partial{L( \mathbf{W} )}}{\partial{u_j}} = \begin{cases}
    \frac{\exp{(u_j)}}{\sum_{j=1}^k \exp^{u_j}}, j* \neq j \\
    \frac{\exp{u_j} }{\sum_{j=1}^k \exp ^{u_j}} - 1, j* = j
\end{cases} \\
&u_j = \mathbf{w_j}^T \mathbf{x} + b_j \\
&u_{j*} = \mathbf{w_{j*}}^T \mathbf{x} + b_{j*} \\
&\text{具体到每个参数的计算：} \\
&\frac{\partial L(\mathbf{W})}{\partial w_{jl}} = \begin{cases}
    y_jx_l, j*\neq j\\
    (y_j - 1)x_l, j* = j
\end{cases}
\end{align*}
$$

随机梯度下降和小批量梯度下降的区别：
- 小批量梯度下降在每个batch上计算各条数据所得梯度的平均值来更新参数
- 随机梯度下降每条数据都会更新参数

https://zhuanlan.zhihu.com/p/72929546

In [95]:
# model

class SoftmaxRegression():
    def __init__(self, num_features, num_class):
        # 随机初始化参数 num_features * num_class num_feature 包含了bias
        ## W num_features * num_class # 这里也需要加入bias，bias = W[-1, :] 1 * num_class
        self.num_features = num_features
        self.num_class = num_class
        self.w = np.random.randn(num_features, num_class)
    
    def forward(self, x):
        """前向传播返回yhat_p，x的数据格式为(num_features) * n，当使用backward时，一次只能传入一组数据，但是当计算loss时，可以一次传入多组数据做前向传播

        Args:
            x (np.array): 为一组数据，格式为 n * num_features ，应当转置为(num_features ) * n

        Returns:
            yhat_p (np.array): 类别概率分布 num_class * n
        """
        x = x.T # (num_features ) * n
        assert x.shape[0] == self.num_features
        # W^T * x = [z1, z2, z3]^T
        temp = np.matmul(self.w.T, x) # num_class * n
        # y_k = p(y = k | x) = exp(z_k) / sum(exp(z_i))
        assert temp.shape == (self.num_class, x.shape[1])
        # 对temp列求和
        sum = np.sum(np.exp(temp), axis=0) # 1 * n
        print(sum.shape)
        # 将sum 转化为 1 * n
        sum = sum.reshape(1, -1)
        assert sum.shape == (1, x.shape[1])
        sum_m = np.repeat(sum, self.num_class, axis=0) # num_class * n
        assert sum_m.shape == (self.num_class, x.shape[1])
        # yhat_p = np.exp(temp) / sum # num_class * 1
        yhat_p = np.divide(np.exp(temp), sum_m) # num_class * n
        assert yhat_p.shape == (self.num_class, x.shape[1])
        return yhat_p
    
    def backward(self, x, yhat_p, y, lr):
        """反向传播，直接更新参数，返回负梯度只是作为log，使用随机梯度下降法

        Args:
            x (输入特征): 输入的一条数据,(num_features) * 1
            yhat_p (预测概率分布): num_class * 1
            y (实际标签值): 1 * 1
        Returns:
            grad (np.array): 负梯度矩阵 num_class * (num_features)
        """
        yhat = np.argmax(yhat_p) # 获取每个样本的预测类别 1 * 1
        # 构建梯度矩阵 grad.shape = (num_class, num_features)
        grad = np.zeros((self.num_class, self.num_features))
        for j in range(self.num_class):
            for l in range(self.num_features):
                if yhat == y:
                    grad[j][l] = (y - 1) * x[l]
                else:
                    grad[j][l] = y * x[l]
        # backward内部更新参数一下是随机梯度更新方式（一次只取一条数据）
        self.w = self.w - lr * grad
        return -grad


In [178]:
# loss 
def loss(model, x, y):
    """计算loss

    Args:
        model (SoftmaxRegression): 模型主要取w
        x (np.array): 输入特征 n * num_features
        y (np.array): 1 * n
    
    Returns:
        loss (np.array): 1 * 1
    """
    print(x.shape)
    y = y.astype(int)
    sum_loss = 0
    for i in range(x.shape[0]):
        sum = 0
        assert x[i].shape   == (x.shape[1],) # (num_features,)
        print(model.w.shape)
        print(model.w[:, y].shape)
        assert model.w[:, y].shape == (x.shape[1], 1) # num_features * 1
        u_j_act = np.matmul(model.w[:, y].T , x[i])
        assert u_j_act.shape == (1,) # 1 * 1
        print(u_j_act.shape)
        # assert 应该为一个数
        assert u_j_act.shape == (1,)
        print(u_j_act.shape)
        for j in range(x.shape[1]):
            u_j = np.matmul(model.w[:, j], x[i])
            sum += np.log(u_j) - u_j_act
        loss = np.log(sum)
        sum_loss += loss
    
    return sum_loss/ x.shape[0]

In [179]:
# train
def train(model, dataset: Dataset, num_epochs, learning_rate = 0.01, random = True):
    file = None
    if random:
        file = open(f"log2/log_e{num_epochs}_lr{learning_rate}_random.csv", "w")
    else:
        file = open(f"log2/log_e{num_epochs}_lr{learning_rate}.csv", "w")
    file.write(f"epoch,loss,test_loss(cost),wight\n")
    cost = -1
    for e in range(num_epochs):
        if random:
            dataset.random()
            #### Compute outputs #### 
            yhat = model.forward(dataset.train_feature)
            num_features = dataset.train_feature.shape[1]
            num_data = dataset.train_feature.shape[0]
            assert yhat.shape == (model.num_class, num_data) # num_class * n
            #### Compute gradients and Update gradients ####
            for i in range(num_data):
                negative_grad_tmp = model.backward(dataset.train_feature[i], yhat[:, i], dataset.train_label[i], learning_rate)
            #### Logging ####
            # 测试集上平均均方误差 
            cost = loss(model, dataset.test_feature, dataset.test_label)
        file.write(f"{e}, {loss(model, dataset.train_feature, dataset.train_label)}, {cost}, {model.w}\n")
        print(f"epoch:{e} | loss:{loss(model,dataset.train_feature, dataset.train_label)} | test_loss(cost):{cost}")
    file.close()
    if random:
        print(f"log_file: log_e{num_epochs}_lr{learning_rate}_random.csv")
    else:
        print(f"log_file: log_e{num_epochs}_lr{learning_rate}.csv")
    return cost   

In [180]:
dataset = Dataset(iris)

X_train = dataset.train_feature
Y_train = dataset.train_label
print(X_train)
print(Y_train)

assert X_train.shape == (120, 3)
model = SoftmaxRegression(num_features=X_train.shape[1], num_class=3)

train(model, dataset, num_epochs=100, learning_rate=0.01, random=True)



[[-2.86624165  0.06936447  1.        ]
 [-2.20948924  0.43666314  1.        ]
 [-2.56231991  0.36771886  1.        ]
 [ 1.04413183  0.2283619   1.        ]
 [-2.59873675  1.09314576  1.        ]
 [ 2.38800302  0.4646398   1.        ]
 [ 0.98493451 -0.12481785  1.        ]
 [ 2.91675097  0.78279195  1.        ]
 [ 1.55780216  0.26749545  1.        ]
 [-0.0087454  -0.72308191  1.        ]
 [ 0.81329065 -0.1633503   1.        ]
 [ 2.31415471  0.18365128  1.        ]
 [ 3.23067366  1.37416509  1.        ]
 [ 0.24595768 -0.2685244   1.        ]
 [-2.63692688 -0.12132235  1.        ]
 [-2.40561449  0.18887143  1.        ]
 [-2.72871654  0.32675451  1.        ]
 [-2.71445143 -0.2502082   1.        ]
 [ 1.41523588 -0.57491635  1.        ]
 [-2.35575405 -0.03728186  1.        ]
 [-0.74912267 -1.00489096  1.        ]
 [ 1.94968906  0.04194326  1.        ]
 [-2.71414169 -0.17700123  1.        ]
 [ 1.22069088  0.40761959  1.        ]
 [ 2.41874618  0.3047982   1.        ]
 [-2.78610927 -0.235112  

AssertionError: 